# Linear Kirchhoff-Love shell

Reference values for free boundary conditions: 

$t=0.1$: $0.1856305$, 

$t=0.01$: $0.1502913$, 

$t=0.001$: $0.1498749$

In [ ]:
from ngsolve import *
import netgen.meshing as meshing
from ngsolve.webgui import Draw

In [ ]:
thickness = 0.1
order = 3

# material properties
E   = 2.85e4
nu  = 0.3
mu  = E/(2*(1+nu))
lam = E*nu/(1-nu**2)
G = E/(2*(1+nu))

free = "right"
symmetry ="left|top|bottom"

# periodic force
zeta = IfPos(y, atan(z/y), pi/2)
e_n = specialcf.normal(3)
force = 1e4*thickness*cos(2*zeta)*e_n*thickness**2

# geometry
mapping = lambda x,y,z : (x,sqrt(1+x**2)*cos(pi/2*y),sqrt(1+x**2)*sin(pi/2*y))
geom = meshing.SurfaceGeometry(mapping)
mesh = Mesh(geom.GenerateMesh(quads=False, nx=5, ny=5)).Curve(order)
Draw(mesh)

In [ ]:
def MaterialStress(mat):
    return E/(1-nu**2)*((1-nu)*mat+nu*Trace(mat)*Id(3))

t = thickness
       
fesU = VectorH1(mesh, order=order, dirichletx_bbnd="left", dirichlety_bbnd="top", dirichletz_bbnd="bottom")
fesB = VectorH1(mesh, order=order,orderface=order+1 if order < 4 else order, \
                dirichletx_bbnd="left", dirichlety_bbnd="top", dirichletz_bbnd="bottom")
fesS = HCurl(mesh,order=order, type1=True)
fes = fesU*fesB*fesS
(u,beta,gamma), (du,dbeta,dgamma) = fes.TnT()
gamma, dgamma = gamma.Operator("dual").Trace(), dgamma.Operator("dual").Trace()


n = specialcf.normal(3)
Ptau = Id(3) - OuterProduct(n,n)
gradu = Grad(u).Trace()
graddu = Grad(du).Trace()


a = BilinearForm(fes, symmetric=True)
a += (t*InnerProduct(MaterialStress(Sym(Ptau*gradu)), Sym(Ptau*graddu)))*ds    
a += (t**3/12*InnerProduct(MaterialStress(Sym(Ptau*Grad(beta).Trace() + gradu.trans*Grad(n))), \
                        Sym(Ptau*Grad(dbeta).Trace() + graddu.trans*Grad(n))))*ds
a += ((gradu.trans*n+beta)*dgamma + (graddu.trans*n+dbeta)*gamma)*ds \
        + ((gradu.trans*n+beta)*dgamma + (graddu.trans*n+dbeta)*gamma)*ds(element_boundary=True)
a += (t*G*(n*beta)*(n*dbeta))*ds
    
f = LinearForm(fes)
f += force*du*ds

In [ ]:
gfsol = GridFunction(fes)
with TaskManager():
    a.Assemble()
    f.Assemble()
    
    gfsol.vec.data = a.mat.Inverse(fes.FreeDofs(),inverse="pardiso")*f.vec

In [ ]:
gfu, gfbeta,gfgamma = gfsol.components
print("value = ", gfu(mesh(0,0,1,BND))[2])
Draw(gfu, mesh, deformation=True)